In [ ]:
%pip install qiskit
%pip install qiskit-ibm-runtime
%pip install qiskit[visualization]
%pip install qiskit_aer

In [ ]:
from math import gcd,log
from random import randint
import numpy as np
from qiskit import *

from qiskit.primitives import BackendSamplerV2
from qiskit.providers.basic_provider import BasicProvider
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Sampler

import RSA_module
import Shor_module

In [ ]:
decrypted_msg = RSA_module.decrypt(encryption_obj, private)

print("\nMensagem decriptada: " + decrypted_msg)

In [ ]:
def modular_inverse(a,m): 
    a = a % m; 
    for x in range(1, m) : 
        if ((a * x) % m == 1) : 
            return x 
    return 1

In [ ]:
N=18727 # 61*307
assert N>0,"Input must be positive"
print(Shor_module.shors_breaker(N))


In [ ]:
bit_length = 4

public, private = RSA_module.generate_keypair(2**bit_length)
print("\nChave pública: " + str(public))
print("Chave privada: " + str(private))

In [ ]:
msg = "Olá mundo!"
encrypted_msg, encryption_obj = RSA_module.encrypt(msg, public)

print("\nMensagem original: " + msg)
print("Mensagem encriptada: " + encrypted_msg)

In [ ]:
N_shor = public[1]
assert N_shor>0,"Input must be positive"
p,q = Shor_module.shors_breaker(N_shor)
# Obtem chave privada
phi = (p-1) * (q-1)  
d_shor = modular_inverse(public[0], phi) 

In [ ]:
print("\nChave pública: " + str(public))
print("Chave privada: " + str(private))
print('N: ' + str(N_shor))
print('p: ' + str(p) + ', q: ' + str(q))

In [ ]:
decrypted_msg = RSA_module.decrypt(encryption_obj, (d_shor,N_shor))

print('\nMensagem quebrada com o algorítmo de Shor: ' + decrypted_msg + "\n")